# Data Analysis Intro 
30 May 2023 \
A. Baskind

# Import Python packages

Most of these packages are standard to Anaconda, and, if you don't have them, can install them easily with `conda install <package name>`. However, `PyCO2SYS` is a bit trickier to install.

To install `PyCO2SYS`, write in the command line `pip install PyCO2SYS`. You *may* need to first activate your conda environment

In [2]:
from matplotlib import pyplot as plt
from matplotlib.dates import DateFormatter
import numpy as np
import xarray as xr
import pandas as pd
import scipy
from datetime import datetime, timedelta
import time
import seaborn
import matplotlib.dates as mdates
import bottleneck as bn
import PyCO2SYS as pyco2
import gsw
import math
import netCDF4 as nc
import requests

from importlib import reload
import warnings
warnings.filterwarnings('ignore')



I have also written a couple Python functions that help with data analysis. I implement them below.

The `%` essentially means run the line as if it were in the command line. `cd` directs the computer to the directory where the Python script is. `run` executes the script. Since the script is only a couple of functions, all this does is give you access to these functions.

In [5]:
# %cd phyto
%run PLT.py

# PLT functions
```Python
def get_hydrocat(start_date, end_date, buoy):
    """
    Retrieve Hydrocat data from Andy Davies' sensors using the API
    Drop 0 or negative pH values
    Change time zone from UTC to EST (-5 hours)
    Convert pH from NBS to total scale using PyCO2SYS
        
    INPUTS:
    - start_date: date as a string, formatted as YYYY-MM-DD
    - end_date: date as a string, formatted as YYYY-MM-DD
    - buoy: Jamestown/620/PLT or 720/Greenwich Bay/BG
        
    RETURNS pandas DataFrame
        
    Add '%cd directory' and '%run PLT.py' to your script, making sure the path is correct
    """ 
```
```Python
def dic_to_uM(dic, S, T):
    """
    This function converts from units of umol/kg to uM [umol/L] by calculating density using an equation of state.
        
    INPUTS:
        - DIC or TA in umol/kg
        - Salinity in PSU
        - Temperature in degrees C
        
    OUTPUTS:
    - DIC or TA in uM
    """
```

Here, I demonstrate using a function from `PLT`.

In [25]:
PLT.dic_to_uM(2022.46, 33.464, 20.9)

2069.6911047255107

In [7]:
df = PLT.get_hydrocat('2022-01-01', '2022-12-30', 'PLT')
df

,TmStamp,hydrocatStart,hydrocatTemperature,hydrocatConductivity,hydrocatDissOxygen,hydrocatSalinity,hydrocatFluorescence,hydrocatTurbidity,hydrocatPH,DateTime,pH total
326,2022-01-04T09:31:15.400Z,None,6.5078,31.0263,9.592,30.7871,7.478,30.272,8.04,2022-01-04 04:31:15.400000,7.934479
331,2022-01-04T10:46:15.400Z,None,6.3631,30.9445,9.663,30.8278,1.844,3.473,8.04,2022-01-04 05:46:15.400000,7.934814
332,2022-01-04T11:01:15.200Z,None,6.3063,30.8235,9.633,30.7459,2.373,15.717,8.04,2022-01-04 06:01:15.200000,7.934773
333,2022-01-04T11:16:15.200Z,None,6.2367,30.7264,9.680,30.7015,3.158,5.010,8.04,2022-01-04 06:16:15.200000,7.934821
334,2022-01-04T11:31:15.200Z,None,6.2593,30.6975,9.698,30.6492,2.170,7.818,8.03,2022-01-04 06:31:15.200000,7.924688
...,...,...,...,...,...,...,...,...,...,...,...
31734,2022-12-29T22:46:29.000Z,12/29/2022 22:45:00,3.9276,27.7203,10.546,29.3688,0.097,64.257,8.12,2022-12-29 17:46:29,8.016546
31735,2022-12-29T23:01:29.000Z,12/29/2022 23:00:00,3.9210,27.7232,10.552,29.3782,0.093,64.640,8.11,2022-12-29 18:01:29,8.006574
31736,2022-12-29T23:16:29.000Z,12/29/2022 23:15:00,3.9183,27.7297,10.567,29.3882,0.123,64.546,8.12,2022-12-29 18:16:29,8.016597
31737,2022-12-29T23:31:29.000Z,12/29/2022 23:30:00,3.9342,27.7421,10.652,29.3883,0.123,65.017,8.13,2022-12-29 18:31:29,8.026570


# Setting up API retrieval from Google Sheets

*Notably I set this API up with my google account. It took forever to figure so I am hoping you can just use my credentials. If they don't work, I'll have a tough task ahead of me haha.*

> Make sure to have installed the necessary packages in your conda environment
>>```
conda activate base
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
pip install gspread oauth2client df2gspread

> Have `credentials.json` in your working directory

> Have `servicecredentials.json` in your working directory

> Share your Google sheet with your service account: `google-sheets@third-faculty-385421.iam.gserviceaccount.com`

> Get spreadsheet key from your Google sheet and identify the worksheet you want

In [8]:
#The scope is always look like this so we did not need to change anything

import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

scope = [
   'https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

# Name of our Service Account Key
google_key_file = 'servicecredentials.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_key_file, scope)
gc = gspread.authorize(credentials)

# Dataset 1: Lab Samples found [here](https://docs.google.com/spreadsheets/d/17FFbtUuhUS4UtxB-OjKIP2wCYJoEAmaW6VaHQPcup9U/edit#gid=0)
This Google Sheet has on `Sheet1` all the relevant samples. It provides
* sample name
* location (PLT for time series site; GB for Greenwich Bay)
* date & time of collection
* surface or bottom
* temperature when TA was measured (degC)
* measured TA (uM)
* temperature when DIC was measured (degC)
* measured DIC (uM)
* salinity (PSU)

Since it is a working document, some cells are empty.

The other sheet in the Google Sheet (`Sample Log`) records all samples collected. It provides
* sample name
* location
* depth
* date and time
* completion of TA and DIC

This information is originally recorded in the lab notebook. I do my best to keep the sheet up-to-date, but it's not perfect.

## Retrieve lab samples from Google API

The `spreadsheet_key` can be founded in any sheet's url after '/d/' and before '/edit': https://docs.google.com/spreadsheets/d/17FFbtUuhUS4UtxB-OjKIP2wCYJoEAmaW6VaHQPcup9U/edit#gid=0.

The `wks_name` is the name of the individual sheet within the workbook. In this workbook, the options are Sheet1 and Sample Log. Since sample data is held on Sheet1, use that for `wks_name`.

In [10]:
spreadsheet_key = '17FFbtUuhUS4UtxB-OjKIP2wCYJoEAmaW6VaHQPcup9U'
wks_name = 'Sheet1'

#Opening the worksheet by using Worksheet ID
workbook = gc.open_by_key(spreadsheet_key)
#Selecting which sheet to pulling the data
sheet = workbook.worksheet(wks_name)

#Pulling the data and transform it to the data frame
values = sheet.get_all_values()
labdata = pd.DataFrame(values[1:], columns = values[0])
labdata.head()

,Sample,Time,Location,depth,TA Temp (degC),TA (uM),DIC Temp (degC),DIC (uM),Salinity,"NOTE: Salinity values are currently either derived from buoy data or estimated to be 30 PSU. Abby will measure salinity in lab/update salinities with in situ observations once the semester is over. Additionally, some of the samples here haven't been run for TA yet. Abby will likely run that in the next couple of weeks."
0,PLT 5/9/22 surf A,5/9/22 7:30,PLT,surface,20.3,2075.47,22,1985.40452,30,
1,PLT 5/9/22 surf B,5/9/22 7:30,PLT,surface,20.3,2076.78,22,1955.71939,30,
2,PLT 4/28/22 A,4/28/22 7:30,PLT,surface,20.9,2078.825,22,1931.63783,30,
3,PLT 4/28/22 B,4/28/22 7:30,PLT,surface,20.9,2078.41,22,1931.62999,30,
4,PLT 5/22/22 bottom,5/22/22 7:30,PLT,bottom,20.9,2138.395,22,1978.53868,30,


## Bit of data clean up

Since I have notes written in some of the columns, we will first select which columns we would like from the dataframe. I've also found that often when pulling from Google Sheets, all the data is written as strings, so here we also convert our data to appropriate data types.

In [12]:
df = labdata[['Sample', 'Time', 'Location', 'depth', 'TA Temp (degC)', 'TA (uM)',
       'DIC Temp (degC)', 'DIC (uM)', 'Salinity']]
df['TA (uM)'] = pd.to_numeric(df['TA (uM)'])
df['TA Temp (degC)'] = pd.to_numeric(df['TA Temp (degC)'])
df['DIC (uM)'] = pd.to_numeric(df['DIC (uM)'])
df['DIC Temp (degC)'] = pd.to_numeric(df['DIC Temp (degC)'])
df['Salinity'] = pd.to_numeric(df['Salinity'])
df["DateTime"] = pd.to_datetime(df["Time"])
df.head()

,Sample,Time,Location,depth,TA Temp (degC),TA (uM),DIC Temp (degC),DIC (uM),Salinity,DateTime
0,PLT 5/9/22 surf A,5/9/22 7:30,PLT,surface,20.3,2075.470,22.0,1985.40452,30.0,2022-05-09 07:30:00
1,PLT 5/9/22 surf B,5/9/22 7:30,PLT,surface,20.3,2076.780,22.0,1955.71939,30.0,2022-05-09 07:30:00
2,PLT 4/28/22 A,4/28/22 7:30,PLT,surface,20.9,2078.825,22.0,1931.63783,30.0,2022-04-28 07:30:00
3,PLT 4/28/22 B,4/28/22 7:30,PLT,surface,20.9,2078.410,22.0,1931.62999,30.0,2022-04-28 07:30:00
4,PLT 5/22/22 bottom,5/22/22 7:30,PLT,bottom,20.9,2138.395,22.0,1978.53868,30.0,2022-05-22 07:30:00


We also need to drop missing data.

In [13]:
# For each index in dataframe df...
for ind in df.index:
    # if TA or DIC at that index is Nan/empty...
    if math.isnan(df['TA (uM)'][ind]) or math.isnan(df['DIC (uM)'][ind]):
        # Drop that index
        df = df.drop(ind)

## Unit conversion

Our lab equipment provides DIC and TA in uM (i.e. umol/l). However, units of umol/kg are more useful, and those are the units used by `PyCO2SYS`. So we must convert units.

$$ \frac{\text{umol}}{\text{L}} \times \frac{\text{L}}{0.001\ \text{m}^3} \times \frac{\text{m}^3}{\text{kg}} = \frac{\text{umol}}{\text{kg}}
$$

$$ \text{DIC} \times \frac{1}{0.001} \times \frac{1}{\rho} $$

I have been solving for density $\rho$ using the `gsw` python package. To install, you may need to install externally similar to how we installed `PyCO2SYS`. I've been using `gsw.density.rho(S, T, p)`, which returns in situ density. We have temperature and salinity already in our data, but we must provide pressure. To do this, I create a new column in our dataframe for pressure and say pressure is 0 everywhere, since these samples are being analyzed in the lab where pressure is only from the atmosphere (the function inherently accounts for atmospheric pressure). Notably, since TA and DIC are generalized measured on different days, we have to use different temperatures and thus different densities for both TA and DIC.

In [14]:
# Pressure
df['P'] = 0

# Density
df['rho_DIC'] = gsw.rho(df['Salinity'], df['DIC Temp (degC)'], df['P'])
df['rho_TA'] = gsw.rho(df['Salinity'], df['TA Temp (degC)'], df['P'])

# Converted DIC and TA
df['DIC (umol/kg)'] = df['DIC (uM)']/df['rho_DIC']/0.001
df['TA (umol/kg)'] = df['TA (uM)']/df['rho_TA']/0.001


df.head()

,Sample,Time,Location,depth,TA Temp (degC),TA (uM),DIC Temp (degC),DIC (uM),Salinity,DateTime,P,rho_DIC,rho_TA,DIC (umol/kg),TA (umol/kg)
0,PLT 5/9/22 surf A,5/9/22 7:30,PLT,surface,20.3,2075.470,22.0,1985.40452,30.0,2022-05-09 07:30:00,0,1020.366971,1020.813221,1945.774977,2033.153526
1,PLT 5/9/22 surf B,5/9/22 7:30,PLT,surface,20.3,2076.780,22.0,1955.71939,30.0,2022-05-09 07:30:00,0,1020.366971,1020.813221,1916.682375,2034.436817
2,PLT 4/28/22 A,4/28/22 7:30,PLT,surface,20.9,2078.825,22.0,1931.63783,30.0,2022-04-28 07:30:00,0,1020.366971,1020.658514,1893.081494,2036.748797
3,PLT 4/28/22 B,4/28/22 7:30,PLT,surface,20.9,2078.410,22.0,1931.62999,30.0,2022-04-28 07:30:00,0,1020.366971,1020.658514,1893.073810,2036.342196
4,PLT 5/22/22 bottom,5/22/22 7:30,PLT,bottom,20.9,2138.395,22.0,1978.53868,30.0,2022-05-22 07:30:00,0,1020.366971,1020.658514,1939.046182,2095.113077


## Calculate pH for lab data

To do this, we use `PyCO2SYS`. The documentation for how to use `PyCO2SYS` is [here](https://pyco2sys.readthedocs.io/en/latest/co2sys_nd/).

In [16]:
results = pyco2.sys(par1=df['TA (umol/kg)'],par2=df['DIC (umol/kg)'],par1_type=1,par2_type=2,salinity = df['Salinity'], temperature = df['DIC Temp (degC)'])
df['pH'] = results['pH']
df.head()

,Sample,Time,Location,depth,TA Temp (degC),TA (uM),DIC Temp (degC),DIC (uM),Salinity,DateTime,P,rho_DIC,rho_TA,DIC (umol/kg),TA (umol/kg),pH
0,PLT 5/9/22 surf A,5/9/22 7:30,PLT,surface,20.3,2075.470,22.0,1985.40452,30.0,2022-05-09 07:30:00,0,1020.366971,1020.813221,1945.774977,2033.153526,7.700875
1,PLT 5/9/22 surf B,5/9/22 7:30,PLT,surface,20.3,2076.780,22.0,1955.71939,30.0,2022-05-09 07:30:00,0,1020.366971,1020.813221,1916.682375,2034.436817,7.793153
2,PLT 4/28/22 A,4/28/22 7:30,PLT,surface,20.9,2078.825,22.0,1931.63783,30.0,2022-04-28 07:30:00,0,1020.366971,1020.658514,1893.081494,2036.748797,7.865098
3,PLT 4/28/22 B,4/28/22 7:30,PLT,surface,20.9,2078.410,22.0,1931.62999,30.0,2022-04-28 07:30:00,0,1020.366971,1020.658514,1893.073810,2036.342196,7.864080
4,PLT 5/22/22 bottom,5/22/22 7:30,PLT,bottom,20.9,2138.395,22.0,1978.53868,30.0,2022-05-22 07:30:00,0,1020.366971,1020.658514,1939.046182,2095.113077,7.890143


For reference, `PyCO2SYS` can take a lot of inputs and produce a lot of outputs. This is what it takes/provides.

In [23]:
list(results.keys())

['par1',
 'par1_type',
 'par2',
 'par2_type',
 'alkalinity',
 'dic',
 'opt_k_bisulfate',
 'opt_k_carbonic',
 'opt_k_fluoride',
 'opt_total_borate',
 'opt_gas_constant',
 'opt_pH_scale',
 'opt_buffers_mode',
 'salinity',
 'temperature',
 'pressure',
 'total_ammonia',
 'total_borate',
 'total_calcium',
 'total_fluoride',
 'total_phosphate',
 'total_silicate',
 'total_sulfate',
 'total_sulfide',
 'peng_correction',
 'gas_constant',
 'total_alpha',
 'total_beta',
 'pressure_atmosphere',
 'pressure_atmosphere_out',
 'pH',
 'pCO2',
 'fCO2',
 'bicarbonate',
 'carbonate',
 'aqueous_CO2',
 'xCO2',
 'alkalinity_borate',
 'hydroxide',
 'alkalinity_phosphate',
 'alkalinity_silicate',
 'alkalinity_ammonia',
 'alkalinity_sulfide',
 'hydrogen_free',
 'revelle_factor',
 'saturation_calcite',
 'saturation_aragonite',
 'pH_total',
 'pH_sws',
 'pH_free',
 'pH_nbs',
 'gamma_dic',
 'beta_dic',
 'omega_dic',
 'gamma_alk',
 'beta_alk',
 'omega_alk',
 'isocapnic_quotient',
 'isocapnic_quotient_approx',
 'psi'

# Dataset 2: SeaFET sensor data

The lab has had a couple issues deploying this sensor, so I have not added any recent SeaFET datasets to the shared drive